##  Wrangling the tweet archive of WeRateDogs

### Gathering

In [730]:
# import necessary packages
import pandas as pd
import requests
import os
import tweepy
import json
import re
import numpy as np

In [110]:
# read tweeter archive file
archive = pd.read_csv('twitter-archive-enhanced-2.csv')

In [68]:
# request image prediction file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open (os.path.join(os.getcwd(), url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [111]:
pred = pd.read_csv('image-predictions.tsv', sep='\t')

In [57]:
consumer_key = 'Your Key'
consumer_secret = 'Your Secret'
access_token = 'Your Token'
access_secret = 'Your Secret'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [65]:
%%time
tweets = list(df_arch['tweet_id'])
tweepy_errors = []
for tw in tweets:
    try:
        tweet = api.get_status(tw, tweet_mode='extended')
        with open(os.path.join(os.getcwd(), 'tweep_json.txt'), mode='a') as file:
            json.dump(tweet, file)
            file.write('\n')
    except tweepy.error.TweepError as e:
        print('Error: {}'.format(tw))
        tweepy_errors.append(tw)

Error: 888202515573088257
Error: 873697596434513921
Error: 872668790621863937
Error: 872261713294495745
Error: 869988702071779329
Error: 866816280283807744
Error: 861769973181624320
Error: 856602993587888130
Error: 851953902622658560
Error: 845459076796616705
Error: 844704788403113984
Error: 842892208864923648
Error: 837366284874571778
Error: 837012587749474308


Rate limit reached. Sleeping for: 683


Error: 829374341691346946
Error: 827228250799742977
Error: 812747805718642688
Error: 802247111496568832
Error: 779123168116150273
Error: 775096608509886464
Error: 771004394259247104
Error: 770743923962707968
Error: 759566828574212096
Error: 754011816964026368


Rate limit reached. Sleeping for: 598


Error: 680055455951884288


Rate limit reached. Sleeping for: 496


CPU times: user 50.1 s, sys: 3.13 s, total: 53.2 s
Wall time: 44min 32s


In [89]:
# list of deleted tweets
print(tweepy_errors)

[888202515573088257, 873697596434513921, 872668790621863937, 872261713294495745, 869988702071779329, 866816280283807744, 861769973181624320, 856602993587888130, 851953902622658560, 845459076796616705, 844704788403113984, 842892208864923648, 837366284874571778, 837012587749474308, 829374341691346946, 827228250799742977, 812747805718642688, 802247111496568832, 779123168116150273, 775096608509886464, 771004394259247104, 770743923962707968, 759566828574212096, 754011816964026368, 680055455951884288]


In [791]:
# read retweets and 'like' counts to data frame
df_status = []
with open(os.path.join(os.getcwd(), 'tweep_json.txt'), mode = 'r') as file:
    for line in file:
        status = json.loads(line) 
        df_status.append({'id': status['id'],
                         'time': status['created_at'],
                         'retweeted': status['retweeted'],
                         'retweets': status['retweet_count'],
                         'likes': status['favorite_count']})

In [792]:
status = pd.DataFrame(df_status, columns = ['id', 'time', 'retweeted', 'retweets', 'likes'])

### Access

In [125]:
# Check tweets archive 
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [126]:
archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1505,691793053716221953,NaN,NaN,2016-01-26 01:21:31 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",We usually don't rate penguins but this one is...,NaN,NaN,NaN,https://vine.co/v/OTTVAKw6YlW,10,10,None,None,None,None,None
1499,692187005137076224,NaN,NaN,2016-01-27 03:26:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a rare Arctic Wubberfloof. Unamused by...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/692187005...,12,10,a,None,None,None,None
250,845098359547420673,NaN,NaN,2017-03-24 02:22:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bungalo. She uses that ...,7.733088e+17,4.196984e+09,2016-09-06 23:56:05 +0000,https://twitter.com/dog_rates/status/773308824...,12,10,Bungalo,None,None,None,None
1405,699323444782047232,NaN,NaN,2016-02-15 20:04:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""I'm bathing the children what do you want?"" ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/699323444...,10,10,None,None,None,None,None
1909,674422304705744896,NaN,NaN,2015-12-09 02:56:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ava. She doesn't understand flowers. 1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674422304...,12,10,Ava,None,None,None,None


In [122]:
archive.shape

(2356, 17)

In [138]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [144]:
# check how many tweets are original
archive.shape[0] - 181

2175

In [149]:
archive.retweeted_status_id.value_counts()

7.757333e+17    1
7.507196e+17    1
6.742918e+17    1
6.833919e+17    1
8.269587e+17    1
               ..
7.848260e+17    1
7.806013e+17    1
8.305833e+17    1
7.047611e+17    1
7.331095e+17    1
Name: retweeted_status_id, Length: 181, dtype: int64

In [137]:
archive.name.value_counts()

None       745
a           55
Charlie     12
Lucy        11
Oliver      11
          ... 
Bobble       1
Tayzie       1
Leonard      1
Ron          1
Kendall      1
Name: name, Length: 957, dtype: int64

In [117]:
archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [118]:
archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [197]:
archive_clean[['rating_denominator','rating_numerator']].describe()

,rating_denominator,rating_numerator
count,2356.000000,2356.000000
mean,10.455433,13.126486
std,6.745237,45.876648
min,0.000000,0.000000
25%,10.000000,10.000000
50%,10.000000,11.000000
75%,10.000000,12.000000
max,170.000000,1776.000000


In [145]:
archive.doggo.value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [146]:
archive.floofer.value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [147]:
archive.pupper.value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [148]:
archive.puppo.value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [123]:
# Access image prediction table
pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [124]:
pred.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1854,841077006473256960,https://pbs.twimg.com/media/C6wbE5bXUAAh1Hv.jpg,1,Brittany_spaniel,0.962985,True,Blenheim_spaniel,0.014820,True,clumber,0.009557,True
1750,823939628516474880,https://pbs.twimg.com/media/C284uD8WgAEmMVn.jpg,1,schipperke,0.234076,True,curly-coated_retriever,0.193093,True,Labrador_retriever,0.095197,True
664,682697186228989953,https://pbs.twimg.com/media/CXltdtaWYAIuX_V.jpg,1,bald_eagle,0.097232,False,torch,0.096621,False,cliff,0.090385,False
1393,767191397493538821,https://pbs.twimg.com/media/CqWcgcqWcAI43jm.jpg,1,patio,0.708665,False,boathouse,0.110056,False,pier,0.039532,False
329,672082170312290304,https://pbs.twimg.com/media/CVO3KodXAAAj1de.jpg,1,hamster,0.132440,False,toy_poodle,0.123962,True,bubble,0.056212,False


In [127]:
pred.shape

(2075, 12)

In [121]:
pred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [151]:
sum(pred.duplicated())

0

In [128]:
pred.p1.value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
Japanese_spaniel        1
sandbar                 1
EntleBucher             1
microwave               1
earthstar               1
Name: p1, Length: 378, dtype: int64

In [129]:
pred.p2.value_counts()

Labrador_retriever    104
golden_retriever       92
Cardigan               73
Chihuahua              44
Pomeranian             42
                     ... 
lampshade               1
umbrella                1
spotted_salamander      1
otter                   1
bannister               1
Name: p2, Length: 405, dtype: int64

In [163]:
# Acess retweet status table
status.head()

,id,time,retweeted,retweets,likes
0,892420643555336193,Tue Aug 01 16:23:56 +0000 2017,False,7745,36427
1,892177421306343426,Tue Aug 01 00:17:27 +0000 2017,False,5737,31368
2,891815181378084864,Mon Jul 31 00:18:03 +0000 2017,False,3798,23640
3,891689557279858688,Sun Jul 30 15:58:51 +0000 2017,False,7902,39740
4,891327558926688256,Sat Jul 29 16:00:24 +0000 2017,False,8539,37932


In [164]:
status.sample(5)

,id,time,retweeted,retweets,likes
3008,786709082849828864,Thu Oct 13 23:23:56 +0000 2016,False,6269,18708
306,834458053273591808,Wed Feb 22 17:41:18 +0000 2017,False,1699,9707
1347,702276748847800320,Tue Feb 23 23:39:59 +0000 2016,False,766,2471
4043,679530280114372609,Wed Dec 23 05:13:38 +0000 2015,False,2075,4800
2178,668641109086707712,Mon Nov 23 04:03:57 +0000 2015,False,524,1038


In [165]:
status.shape

(4662, 5)

In [166]:
sum(status.duplicated())

1761

In [167]:
status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4662 entries, 0 to 4661
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         4662 non-null   int64 
 1   time       4662 non-null   object
 2   retweeted  4662 non-null   bool  
 3   retweets   4662 non-null   int64 
 4   likes      4662 non-null   int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 150.4+ KB


In [168]:
status.describe()

,id,retweets,likes
count,4.662000e+03,4662.000000,4662.000000
mean,7.419079e+17,2720.601673,7608.676105
std,6.822438e+16,4601.338496,11803.806877
min,6.660209e+17,1.000000,0.000000
25%,6.782612e+17,548.750000,1328.000000
50%,7.182469e+17,1277.000000,3312.000000
75%,7.986712e+17,3159.250000,9329.000000
max,8.924206e+17,78330.000000,157338.000000


In [169]:
status.retweeted.value_counts()

False    4662
Name: retweeted, dtype: int64

#### Quality

`archive` table:
- Missing information(name for dogs, retweeted infomation etc.)
- Errorness datatype(timestamp datatype is object, retweet_id datatype is float)
- Unsual values for rating numerators and denominators
- Wierd name for some dog names('a' appears 55 times)
- Lowercase for some dog names and uppercase for others

`prediction` table:
- Missing information(2175 original tweets in the archive table but only 2075 in prediction table)
- Lowercase for some dog breed and uppercase for others(Labrador_retriever, golden_retriever etc.)

`status` table:
- Duplicated entries(1761)
- Missing values(occured during aquiring, some tweets are deleted)
- Errorness datatype(object for time instead of datetime)

#### Tidiness

- Tables have duplicated columns(id)
- `prediction` table can be condensed for evaluation
- `archive` table dog stages should be combined in one column 

### Cleaning

In [793]:
# Copy dataframe
archive_clean = archive.copy()
pred_clean = pred.copy()
status_clean = status.copy()

**Clean unnecessary data for `archive` table**

In [794]:
col = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
archive_clean = archive_clean.drop(col, axis=1)

In [795]:
archive_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


**Fix errorness datatype in `archive` table**

In [796]:
archive_clean.timestamp = pd.to_datetime(archive_clean.timestamp)

In [797]:
archive_clean.dtypes

tweet_id                            int64
timestamp             datetime64[ns, UTC]
source                             object
text                               object
expanded_urls                      object
rating_numerator                    int64
rating_denominator                  int64
name                               object
doggo                              object
floofer                            object
pupper                             object
puppo                              object
dtype: object

**Standardize ratings from numerators and denominators**

In [798]:
# Recall the statistics of ratings
archive_clean[['rating_denominator','rating_numerator']].describe()

,rating_denominator,rating_numerator
count,2356.000000,2356.000000
mean,10.455433,13.126486
std,6.745237,45.876648
min,0.000000,0.000000
25%,10.000000,10.000000
50%,10.000000,11.000000
75%,10.000000,12.000000
max,170.000000,1776.000000


In [799]:
# Find decimal values
twitter_txt = []
inds = []
rating = []
for i, text in archive_clean['text'].iteritems():
    if bool(re.search('\d+\.\d+\/\d+', text)):
        twitter_txt.append(text)
        inds.append(i)
        rating.append(re.findall('\d+\.\d+\/\d+', text)[0])
# Print ratings with decimals        
print(twitter_txt)

['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948', "RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…", "This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS", "This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq", "I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace", 'Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD']


In [800]:
# Replace the misplaced value with extracted value
for i, ind in enumerate(inds):
    archive_clean.iloc[ind, 5] = round(float(rating[i].split('/')[0]), 1)

In [801]:
# Remove denominators are not 10 since 10 is the standard value
archive_clean = archive_clean[archive_clean.rating_denominator == 10]

In [802]:
# Create ratings column
archive_clean['ratings'] = archive_clean.rating_numerator/10

In [803]:
archive_clean.rating_numerator.value_counts()

12.0      558
11.0      463
10.0      461
13.0      351
9.0       156
8.0       102
14.0       54
7.0        54
5.0        35
6.0        32
3.0        19
4.0        15
2.0         9
1.0         8
15.0        2
9.8         2
0.0         2
420.0       2
11.3        2
9.5         1
666.0       1
17.0        1
13.5        1
182.0       1
1776.0      1
Name: rating_numerator, dtype: int64

In [804]:
# Remove invalid values, numerator greater than 100
archive_clean = archive_clean[archive_clean.rating_numerator <= 100]

In [805]:
archive_clean.describe()

,tweet_id,rating_numerator,rating_denominator,ratings
count,2.328000e+03,2328.000000,2328.0,2328.000000
mean,7.428180e+17,10.712285,10.0,1.071229
std,6.864100e+16,2.172597,0.0,0.217260
min,6.660209e+17,0.000000,10.0,0.000000
25%,6.783704e+17,10.000000,10.0,1.000000
50%,7.198478e+17,11.000000,10.0,1.100000
75%,7.993373e+17,12.000000,10.0,1.200000
max,8.924206e+17,17.000000,10.0,1.700000


In [806]:
# Use rating as standardize rating system
archive_clean = archive_clean.drop(['rating_numerator', 'rating_denominator'], axis = 1)

**Change invalid dog names and use uniformed cases** 

In [807]:
# Replace all letters and numbers name
archive_clean.name = archive_clean.name.str.replace('^[a-z]+', 'None').str.title()

In [808]:
archive_clean.name.value_counts()

None       830
Charlie     12
Cooper      11
Lucy        11
Oliver      11
          ... 
Leonard      1
Ron          1
Tyrus        1
Bilbo        1
Odin         1
Name: name, Length: 930, dtype: int64

In [809]:
archive_clean.shape

(2328, 11)

**Change dog name to uniformed style in prediction table**

In [810]:
cols = ['p1', 'p2', 'p3']
for col in cols:
    pred_clean[col] = pred_clean[col].apply(lambda x: x.replace('_', ' ').title())

In [811]:
pred_clean[cols].head()

,p1,p2,p3
0,Welsh Springer Spaniel,Collie,Shetland Sheepdog
1,Redbone,Miniature Pinscher,Rhodesian Ridgeback
2,German Shepherd,Malinois,Bloodhound
3,Rhodesian Ridgeback,Redbone,Miniature Pinscher
4,Miniature Pinscher,Rottweiler,Doberman


**Remove duplicated entries in status table**

In [812]:
status_clean = status_clean.drop(['retweeted', 'time'], axis = 1)

In [814]:
status_clean = status_clean.drop_duplicates()

In [838]:
status_clean = status_clean[status_clean.id.duplicated() == False]

In [839]:
status_clean.duplicated().any()

False

**Melt dog stages into one column**

In [818]:
# Methods to extract dog stage
def match_stage(x):
    styles = np.array(x.split(','))
    u_sty = np.unique(styles)
    if len(u_sty) > 1:
        return u_sty[1]
    else:
        return u_sty[0]

In [819]:
# Concatenate dog stage columns
temp = archive_clean.copy()
temp['dog_stage'] = temp[['doggo', 'floofer', 'pupper', 'puppo']].apply(lambda x: ','.join(x), axis=1)
temp = temp.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis = 1)
# Extract dog_stage 
temp['dog_stage'] = temp['dog_stage'].apply(lambda x: match_stage(x))

In [820]:
archive_clean = temp

In [821]:
archive_clean.dog_stage.value_counts()

None       1948
pupper      245
doggo        97
puppo        29
floofer       9
Name: dog_stage, dtype: int64

**Remove duplicated columns and merge tables**

In [822]:
# Since second prediction for dog image usually has much lower coef, only keep p1 for analysis
pred_clean = pred_clean.iloc[:,0:5]

In [823]:
# Merge image predication and archive table by twitter id
df_merge1 = archive_clean.merge(pred_clean, on = 'tweet_id', how = 'inner')

In [840]:
# Merge above table with status table
df_merge = df_merge1.merge(status_clean, left_on = 'tweet_id', right_on = 'id', how = 'inner')

In [841]:
df_merge.head()

,tweet_id,timestamp,source,text,expanded_urls,name,ratings,dog_stage,jpg_url,img_num,p1,p1_conf,id,retweets,likes
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,1.3,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,Orange,0.097049,892420643555336193,7745,36427
1,892177421306343426,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,1.3,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,892177421306343426,5737,31368
2,891815181378084864,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,1.2,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,891815181378084864,3798,23640
3,891689557279858688,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,1.3,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,Paper Towel,0.170278,891689557279858688,7902,39740
4,891327558926688256,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,1.2,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,Basset,0.555712,891327558926688256,8539,37932


In [842]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2039 entries, 0 to 2038
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   tweet_id       2039 non-null   int64              
 1   timestamp      2039 non-null   datetime64[ns, UTC]
 2   source         2039 non-null   object             
 3   text           2039 non-null   object             
 4   expanded_urls  2039 non-null   object             
 5   name           2039 non-null   object             
 6   ratings        2039 non-null   float64            
 7   dog_stage      2039 non-null   object             
 8   jpg_url        2039 non-null   object             
 9   img_num        2039 non-null   int64              
 10  p1             2039 non-null   object             
 11  p1_conf        2039 non-null   float64            
 12  id             2039 non-null   int64              
 13  retweets       2039 non-null   int64            

In [843]:
# Save clened dataframe
df_merge.to_csv('twitter_archive_master.csv', encoding = 'utf-8')